# Level 4: RAG Agent

This notebook presents an example of executing queries with a RAG agent in Llama Stack. It shows how to initialize an agent with the RAG tool provided by Llama Stack and to invoke it such that retrieval from a vector DB is activated only when necessary. The notebook also covers document ingestion using the RAG tool.

For a simple (non-agentic) RAG tutorial, please refer to [Level1_simple_RAG.ipynb](./Level1_simple_RAG.ipynb).

## Overview

This notebook covers the following steps:

1. Connecting to a llama stack server.
2. Indexing a collection of documents into a vector DB for later retrieval.
3. Initializing the agent capable of retrieving content from vector DB via tool use.
4. Launching the agent and using it to answer user queries about the documents.


## Prerequisites

Before starting, ensure you have the following:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook. 
- Llama Stack server should be using milvus as its vector DB provider.


## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [ ]:
import uuid

from llama_stack_client import Agent, RAGDocument
from llama_stack_client.lib.agents.event_logger import EventLogger

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [ ]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Finally, we will create a unique name for the document collection to be used for RAG ingestion and retrieval.

In [ ]:
vector_db_id = f"test_vector_db_{uuid.uuid4()}"

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle the fetching, conversion and chunking of the documents' content automatically.

In [ ]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=os.getenv("VDB_EMBEDDING"),
    embedding_dimension=int(os.getenv("VDB_EMBEDDING_DIMENSION", 384)),
    provider_id=os.getenv("VDB_PROVIDER"),
)

# ingest the documents into the newly created document collection
urls = [
    ("https://www.openshift.guide/openshift-guide-screen.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512)),
)

## 3. Executing queries via the RAG agent
- Initialize an agent with a list of tools including the built-in RAG tool. The RAG tool specification must include a list of document collection IDs to retrieve from.
- For each prompt, initialize a new agent session, execute a turn during which a retrieval call may be requested, and output the reply received from the agent.

In [ ]:
queries = [
    "How to install OpenShift?",
]

# initializing the agent
agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant. You must use the knowledge search tool to answer user questions.",
    sampling_params=sampling_params,
    # we make our agent aware of the RAG tool by including builtin::rag/knowledge_search in the list of tools
    tools=[
        dict(
            name="builtin::rag",
            args={
                "vector_db_ids": [vector_db_id],  # list of IDs of document collections to consider during retrieval
            },
        )
    ],
)

for prompt in queries:
    cprint(f"\nUser> {prompt}", "blue")
    
    # create a new turn with a new session ID for each prompt
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=agent.create_session(f"rag-session_{uuid.uuid4()}"),
        stream=stream,
    )
    
    # print the response, including tool calls output
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)

## Key Takeaways
This notebook demonstrated how to implement a RAG agent with Llama Stack. We did this by creating an agent and giving it access to the builtin RAG tool, then invoking the agent on the specified query.

Now let's move on to discuss another type of tool we can use to further enhance our models capabilities, MCP Servers, in the next notebook: [Agents and MCP Servers](./Level5_agents_and_mcp.ipynb)

#### Any Feedback?

If you have any feedback on this or any other notebook in this demo series we'd love to hear it! Please go to https://www.feedback.redhat.com/jfe/form/SV_8pQsoy0U9Ccqsvk and help us improve our demos. 